In [11]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta, date

In [12]:
%pip install openpyxl
import openpyxl

Note: you may need to restart the kernel to use updated packages.


In [13]:
def dataJoin(df : pd.DataFrame) -> pd.DataFrame:
    """
    The function uses data in the form of a menu, where the first column is the pizza type and the second column are
    its ingredients. We implement this to the original dataset of pizzas. This function was essential as the original
    menu had only 4 pizzas, therefore, more variety was needed.    
    """

    data = {
        'pizza_category': ['Margherita', 'Pepperoni', 'Hawaiian', 'BBQ Chicken', 'Vegetarian',
                    'Meat Feast', 'Four Cheese', 'Mushroom', 'Buffalo Chicken',
                    'Mediterranean', 'Supreme', 'Pesto Chicken', 'Vegan', 'Capricciosa',
                    'Truffle'],
        'pizza_ingredients': [
            ['Tomato sauce', 'Mozzarella cheese', 'Basil'],
            ['Tomato sauce', 'Mozzarella cheese', 'Pepperoni'],
            ['Tomato sauce', 'Mozzarella cheese', 'Ham', 'Pineapple'],
            ['BBQ sauce', 'Mozzarella cheese', 'Grilled chicken', 'Red onions', 'Cilantro'],
            ['Tomato sauce', 'Mozzarella cheese', 'Mushrooms', 'Bell peppers', 'Black olives', 'Onions', 'Spinach'],
            ['Tomato sauce', 'Mozzarella cheese', 'Pepperoni', 'Sausage', 'Bacon', 'Ground beef'],
            ['Tomato sauce', 'Mozzarella cheese', 'Cheddar cheese', 'Parmesan cheese', 'Gorgonzola cheese'],
            ['Tomato sauce', 'Mozzarella cheese', 'Mushrooms'],
            ['Buffalo sauce', 'Mozzarella cheese', 'Grilled chicken', 'Red onions'],
            ['Tomato sauce', 'Mozzarella cheese', 'Black olives', 'Feta cheese', 'Spinach', 'Sun-dried tomatoes'],
            ['Tomato sauce', 'Mozzarella cheese', 'Pepperoni', 'Sausage', 'Bell peppers', 'Onions', 'Black olives'],
            ['Pesto sauce', 'Mozzarella cheese', 'Grilled chicken', 'Sun-dried tomatoes', 'Artichokes', 'Pine nuts'],
            ['Tomato sauce', 'Vegan cheese', 'Mushrooms', 'Bell peppers', 'Black olives', 'Onions', 'Spinach'],
            ['Tomato sauce', 'Mozzarella cheese', 'Ham', 'Mushrooms', 'Artichokes', 'Black olives', 'Oregano'],
            ['White truffle oil', 'Mozzarella cheese', 'Mushrooms', 'Parmesan cheese', 'Arugula']
        ]
    }

    additional_df = pd.DataFrame(data)
    additional_df
    addi = [df, additional_df]
    df = pd.concat(addi, ignore_index = True)
    return df

### All types of Pizza and Ingredients

In [14]:

def pizzaAndIngredients(df : pd.DataFrame) -> pd.DataFrame:

    """
    This function takes in the dataframe and creates two different dataframes which account for the variety
    of pizzas and the variety of ingredients. The ingredients dataframe has the ingredient, the pizza that 
    uses this ingredient.
    """
    all_ingredients = {}
    all_pizzas = {}

    # iterate over orders
    for index, order in df.iterrows():
        pizza = df.iloc[index, 0]
        ingredients = df.iloc[index, 1]

        # all things the pizza is made out of
        for product in ingredients:
            if product not in all_ingredients:
                all_ingredients[product] = [pizza]
            else:
                if pizza not in all_ingredients[product]:
                    # Convert DataFrame to list before appending pizza
                    if isinstance(all_ingredients[product], pd.DataFrame):
                        all_ingredients[product] = all_ingredients[product]['Pizzas'].tolist()
                    all_ingredients[product].append(pizza)

        if pizza not in all_pizzas:
            all_pizzas[pizza] = ingredients

    all_ingredients = pd.DataFrame(list(all_ingredients.items()), columns=['Ingredients', 'Pizzas'])
    return all_ingredients, all_pizzas


In [15]:
def expiryGen(all_ingredients : pd.DataFrame) -> pd.DataFrame:
    """
    This function appends the expiry date which is generated randomly over as a date over the next 7 days, and adds 
    a column for the expiry date.
    """
    #today
    today = date.today()

    all_ingredients["Expiry Date"] = None
    for index, row in all_ingredients.iterrows():

        #random range
        random_days = random.randint(0, 7)

        #assigning random day over next 7 days
        all_ingredients.iloc[index,2]= today + timedelta(days=random_days)

    return all_ingredients



In [16]:
def recommend(all_ingredients : pd.DataFrame, expiry : list) -> list:
    """ 
    We use this function as a recommendation system.
    """
    recommendation = []
    for i in range(len(expiry)):
        if expiry[i] == True:
            if len(all_ingredients.iloc[i, 1]) >= 3:
                continue
            for pizza in all_ingredients.iloc[i, 1]:
                if pizza not in recommendation:
                    recommendation.append(pizza)
    
    return recommendation


In [17]:
def main():
    df = pd.read_excel("./pacos pizza dataset.xlsx")
    df = df.drop(df.iloc[:, 0:8],axis = 1)
    df['pizza_ingredients'] = df['pizza_ingredients'].apply(lambda x: [ingredient.strip() for ingredient in x.split(',')])
    df = dataJoin(df)
    all_ingredients, all_pizzas = pizzaAndIngredients(df)
    all_ingredients = expiryGen(all_ingredients)
    almost_expired = all_ingredients['Expiry Date'] - date.today() <= pd.Timedelta(days = 1)
    
    # print(recommend(all_ingredients, almost_expired))
    
    #setting seed for controlled random result
    random.seed(9933)
    return recommend(all_ingredients, almost_expired)

In [20]:
main()

['Classic', 'BBQ Chicken', 'Buffalo Chicken', 'Pesto Chicken']